### Lab 3.1: Basic Neural Network in PyTorch - Solution

Let's create a linear classifier one more time, but using PyTorch's automatic differentiation and optimization algorithms.  Then you will extend the perceptron into a multi-layer perceptron (MLP).

In [1]:
import numpy as np
import torch

We need to explicitly tell PyTorch when creating a tensor that we are interested in later computing its gradient

In [2]:
a = torch.tensor(5.,requires_grad=True)
a

tensor(5., requires_grad=True)

In [3]:
b = torch.tensor(6.,requires_grad=True)
c = 2*a+3*b
c

tensor(28., grad_fn=<AddBackward0>)

To extract the gradients, we first need to call `backward()`.

In [4]:
c.backward()

Now to get the gradient of any variable with respect to `c`, we simply access the `grad` attribute of that variable.

In [5]:
a.grad

tensor(2.)

In [6]:
b.grad

tensor(3.)

Let's load and format the Palmer penguins dataset for multi-class classification.

In [7]:
from palmerpenguins import load_penguins
from matplotlib import pyplot as plt

In [8]:
df = load_penguins()

# drop rows with missing values
df.dropna(inplace=True)

# get two features
X = df[['flipper_length_mm','bill_length_mm']].values

# convert species labels to integers
y = df['species'].map({'Adelie':0,'Chinstrap':1,'Gentoo':2}).values

To make the learning algorithm work more smoothly, we we will subtract the mean of each feature.

Here `np.mean` calculates a mean, and `axis=0` tells NumPy to calculate the mean over the rows (calculate the mean of each column).

In [9]:
X -= np.mean(X,axis=0)

Now we will convert our `X` and `y` arrays to torch Tensors.

In [10]:
X = torch.tensor(X).float()
y = torch.tensor(y).long()

In [11]:
from torch import nn

The `torch.nn.Sequential` class creates a feed-forward network from a list of `nn.Module` objects.  Here we provide a single `nn.Linear` class which performs an affine transformation ($Wx+b$) so that we will have a linear classifier.

In [12]:
linear_model = torch.nn.Sequential(
    torch.nn.Linear(2,3), # two inputs, three outputs
)

Now we create a cross-entropy loss function object and a stochastic gradient descent (SGD) optimizer.

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
lr = 1e-2
opt = torch.optim.SGD(linear_model.parameters(), lr=lr)

Finally we can iteratively optimize the model.

In [15]:
epochs = 100
for epoch in range(epochs):
    opt.zero_grad() # zero out the gradients

    z = linear_model(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 0.5305097699165344
epoch 1: loss is 0.5094084143638611
epoch 2: loss is 0.4900461435317993
epoch 3: loss is 0.4723173975944519
epoch 4: loss is 0.45609423518180847
epoch 5: loss is 0.44124436378479004
epoch 6: loss is 0.42764177918434143
epoch 7: loss is 0.415169894695282
epoch 8: loss is 0.4037216305732727
epoch 9: loss is 0.39319851994514465
epoch 10: loss is 0.38350945711135864
epoch 11: loss is 0.3745700418949127
epoch 12: loss is 0.36630210280418396
epoch 13: loss is 0.3586336076259613
epoch 14: loss is 0.35149848461151123
epoch 15: loss is 0.3448368012905121
epoch 16: loss is 0.3385946452617645
epoch 17: loss is 0.33272409439086914
epoch 18: loss is 0.3271826505661011
epoch 19: loss is 0.3219333291053772
epoch 20: loss is 0.31694409251213074
epoch 21: loss is 0.312187135219574
epoch 22: loss is 0.307638943195343
epoch 23: loss is 0.3032791018486023
epoch 24: loss is 0.29909050464630127
epoch 25: loss is 0.2950584888458252
epoch 26: loss is 0.29117056727409363
epo

### Exercises

Extend the above code to implement an MLP with a single hidden layer of size 100.

Write code to compute the accuracy of each model.

Can you get the MLP to outperform the linear model?

In [16]:
mlp = torch.nn.Sequential(
    torch.nn.Linear(2,100),
    torch.nn.ReLU(),
    torch.nn.Linear(100,3),
)

opt = torch.optim.SGD(mlp.parameters(), lr=lr)

for epoch in range(epochs):
    opt.zero_grad() # zero out the gradients

    z = mlp(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 4.431340217590332
epoch 1: loss is 0.4677637815475464
epoch 2: loss is 0.2965518534183502
epoch 3: loss is 0.2327469438314438
epoch 4: loss is 0.2164786159992218
epoch 5: loss is 0.2075054794549942
epoch 6: loss is 0.2003447413444519
epoch 7: loss is 0.19435431063175201
epoch 8: loss is 0.1891857236623764
epoch 9: loss is 0.18464040756225586
epoch 10: loss is 0.18059131503105164
epoch 11: loss is 0.17695100605487823
epoch 12: loss is 0.17365314066410065
epoch 13: loss is 0.17064611613750458
epoch 14: loss is 0.1678888201713562
epoch 15: loss is 0.16534817218780518
epoch 16: loss is 0.16299712657928467
epoch 17: loss is 0.1608133614063263
epoch 18: loss is 0.15877841413021088
epoch 19: loss is 0.1568765938282013
epoch 20: loss is 0.15509499609470367
epoch 21: loss is 0.15342234075069427
epoch 22: loss is 0.15184913575649261
epoch 23: loss is 0.15036706626415253
epoch 24: loss is 0.1489691585302353
epoch 25: loss is 0.14764906466007233
epoch 26: loss is 0.146401405334472

In [17]:
# Compute the accuracy of each model

def accuracy(y_pred,y_true):
    return (y_pred == y_true).float().mean()

y_pred = linear_model(X).argmax(dim=1)
acc = accuracy(y_pred,y)
print(f'linear model accuracy: {acc.item()}')

y_pred = mlp(X).argmax(dim=1)
acc = accuracy(y_pred,y)
print(f'mlp accuracy: {acc.item()}')

linear model accuracy: 0.9429429173469543
mlp accuracy: 0.954954981803894


MLP has outperformed the linear model with it having an accuracy of 0.952 while the linear model accuracy is 0.949.